## Q. 왜 정확도가 아니라 손실함수를 계산하는가?

- 정확도: 맞은 추론 / 전체 추론
- 손실함수: 추론 결과와 정답 스칼라의 차이를 특정한 방법으로 계산한 것

가중치를 미세하게 조정하면?  
- 정확도는 변동이 적거나 비연속적으로 변화함.
- 손실함수: 유의미한 변화가 연속적으로 생김.

미분하면?
- 정확도: 많은 경우에 0이 나옴
- 의미있는 지표가 나옴

## 오차제곱합

$E = \frac{1}{2} \sum_{k=1}^{n} (y_k - t_k)^2$

$y_k$는 신경망의 출력, $t_k$는 정답 레이블, $k$는 데이터의 차원 수

In [3]:
import numpy as np

def sum_squares_error(y, t):
    return 0.5 * np.sum((y-t)**2)

t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # 정답 2

# case 1. '2'일 확률이 높다고 신경망이 추정했을 때
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
e1 = sum_squares_error(np.array(y), np.array(t))
print(e1)

# case 2. '7'일 확률이 높다고 신경망이 추정했을 때
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
e2 = sum_squares_error(np.array(y), np.array(t))
print(e2)

0.09750000000000003
0.5975


## 교차 엔트로피 오차

$E = - \sum_{k=1}^{n} t_k \log(y_k)$

\* 여기서 log는 밑이 e인 자연로그

In [4]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

# case 1. '2'일 확률이 높다고 신경망이 추정했을 때
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
e1 = cross_entropy_error(np.array(y), np.array(t))
print(e1)

# case 2. '7'일 확률이 높다고 신경망이 추정했을 때
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
e2 = cross_entropy_error(np.array(y), np.array(t))
print(e2)

0.510825457099338
2.302584092994546


## N개의 데이터에 대한 평균 손실 함수 구하기

$E = - \frac{1}{N} \sum_{n=1}^{N} \sum_{k=1}^{n} t_{nk} \log(y_{nk})$

In [ ]:
def cross_entropy_error_batch(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size) # [a, b, c] -> [[a, b, c]]
        y = y.reshape(1, y.size)
    
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

def cross_entropy_error_batch_when_t_label(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size) # 필요한가?
        y = y.reshape(1, y.size)
    
    batch_size = y.shape[0]
    return -np.sum(
        np.log(y[np.arange(batch_size), t] + 1e-7)
    ) / batch_size

    # np.arange(batch_size)
    # batch_size가 5면, [0, 1, 2, 3, 4] 반환

    # y[np.arange(batch_size), t] + 1e-7
    # y[ np.array([0, 1, 2, 3, 4]), t ]
    # t가 [2, 7, 0, 9, 4]라고 하면,
    # y[0, 2] y[1, 7] y[2, 0] y[3, 9] y[4, 4]와 같이 각 데이터에서의 정답 값들만 가져오게 됨
    # CEE에서는 어짜피 다른 값들은 0에 가까운 작은 수들이므로 필요 없음